In [1]:
#################################           Load libs                      #############################################
import os
import sys
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append(os.path.dirname(os.getcwd()))

from src.model.model import Model
%matplotlib inline

from src.utils import metrics_eval, train_test_pred_plot

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Import targets
target = pd.read_csv("../data/series_g.csv", sep=";").series_g.values
# target = pd.read_csv("../data/monthly-beer-production-in-austr.csv", sep=",").iloc[:, -1].values
# target = pd.read_csv("../data/Electric_Production.csv", sep=",").iloc[:, -1].values
# target = pd.read_csv("../data/Chicago_hotels.csv", sep="\t").iloc[:, 3].values
# target = pd.read_csv("../data/PJME_hourly.csv", sep=",").PJME_MW.values

# Generate factors
# f1 = np.arange(1, len(target) + 1).reshape(-1, 1)
# f2 = (np.arange(1, len(target) + 1) ** 2).reshape(-1, 1)
# factors = np.hstack((f1, f2))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(factors, target, test_size=12, random_state=42, shuffle=False)
y_train, y_test = train_test_split(target, test_size=12, shuffle=False)

configs ={"model": {
            "layers": [
                {
                    "type": "LSTM",
                    "neurons": 128,
                    "return_sequences": False
                },
                {
                    "type": "Dropout",
                    "rate": 0.2
                },
                {
                    "type": "Dense",
                    "activation": "linear"
                }
            ]
        }}

In [ ]:
model = Model(configs=configs,
              n_step_in=12,
              n_step_out=len(y_test),
              test_len=len(y_test),
              # n_features=X_train.shape[1]
              loss="mae",
              optimizer="adam"
             )
pprint(model.params)
model.model.summary()

In [ ]:
try:
    os.chdir("../reports")
except:
    os.makedirs("../reports")
    os.chdir("../reports")
new_folder_num = str(len(os.listdir()) + 1)
new_folder = "%s/%s" % ("../reports", new_folder_num)
os.mkdir(new_folder)

In [ ]:
from tensorflow.keras import callbacks
from sklearn.preprocessing import MinMaxScaler

my_callbacks = [callbacks.EarlyStopping(patience=30,
                                     monitor='val_loss')]
my_scaler = MinMaxScaler(feature_range=(0, 1))

model.fit(
          # factors=factors,
          target=target,
          epochs=100,
          batch_size=12,
          callbacks = my_callbacks,
          scaler=my_scaler,
          save_dir=new_folder,
          verbose=2)

In [ ]:
predicted = model.predict(
    # factors=X_train[-model.n_step_in:],
    target=y_train[-model.n_step_in:],
    prediction_len=len(y_test)
)

metrics_eval(y_test, predicted, print_result=True, save_dir=new_folder);
train_test_pred_plot(train=y_train, test=y_test, predicted=predicted, save_dir=new_folder)